In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings

np.random.seed(0)

warnings.filterwarnings('ignore')
%matplotlib inline

Реализация класса ```LinearRegressionSGD``` c обучением и и применением линейной регрессии, построенной с помощью стохастического градиентного спуска.

In [2]:
from sklearn.base import BaseEstimator

class LinearRegressionSGD(BaseEstimator):
    def __init__(self, epsilon=1e-4, max_steps=100, w0=None, alpha=1e-4):
        """
        epsilon: разница для нормы изменения весов
        max_steps: максимальное количество шагов в градиентном спуске
        w0: np.array (d,) - начальные веса
        alpha: шаг обучения
        """
        self.epsilon = epsilon
        self.max_steps = max_steps
        self.w0 = w0
        self.alpha = alpha
        self.w = None
        self.w_history = []

    def fit(self, X, y):
        """
        X: np.array (l, d)
        y: np.array (l)
        ---
        output: self
        """
        l, d = X.shape

        if self.w0 is None:
          self.w0 = np.zeros(d)

        self.w = self.w0

        for step in range(self.max_steps):
          self.w_history.append(self.w)

          w_new = self.w - self.alpha * self.calc_gradient(X, y)

          if (np.linalg.norm(w_new - self.w) < self.epsilon):
            break

          self.w = w_new

        return self
    def predict(self, X):
        """
        X: np.array (l, d)
        ---
        output: np.array (l)
        """
        if self.w is None:
            raise Exception('Not trained yet')

        l, d = X.shape

        y_pred = []

        for i in range(l):
          y_pred.append(np.dot(X[i], self.w))

        return np.array(y_pred)
        return y_pred

    def calc_gradient(self, X, y):
        """
        X: np.array (l, d)
        y: np.array (l)
        ---
        output: np.array (d)
        """

        l, d = X.shape
        gradient = []

        for j in range(d):
          dQ = 0
          i = np.random.choice(range(l))
          dQ += (2/l) * X[i][j] * (np.dot(X[i], self.w) - y[i])
          gradient.append(dQ)

        return np.array(gradient)

Проверять работу мы будем на имеющемся в sklearn наборе данных boston: в нём нужно по информации о доме предсказать его стоимость.

In [ ]:
!pip install scikit-learn==1.1.3

In [3]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

data = load_boston()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

X_train, X_test, y_train, y_test = train_test_split(np.array(X), y, test_size=0.3, random_state=10)

Метрикой качества будет MAPE - Mean Absolute Percentage Error.

In [4]:
def MAPE(y_true, y_pred):
    """
        y_true: np.array (l)
        y_pred: np.array (l)
        ---
        output: float [0, +inf)
    """
    mape = np.mean(np.abs((y_true - y_pred) / y_true))*100

    return mape

In [5]:
y_0 = np.full(y_test.shape, np.mean(y_test))

In [ ]:
y_0

In [7]:
MAPE(y_test, y_0)

37.41588297684096

Обучение ```LinearRegressionSGD``` с базовыми параметрами на тренировочном наборе данных (```X_train```, ```y_train```)

In [8]:
sgd = LinearRegressionSGD()
sgd.fit(X_train, y_train)

y_pred_sgd = sgd.predict(X_test)

MAPE(y_test, y_pred_sgd)

39.31827706027169

Вычисление весов по точной формуле, используя ```X_train``` и ```y_train```.

In [9]:
w_true = np.linalg.inv(X_train.T @ X_train) @ X_train.T @ y_train
y_pred_lr = np.dot(X_test, w_true)

MAPE(y_test, y_pred_lr)

18.95313481637632

Чтобы избежать переобучения, предлагается добавить к оптимизируемому функционалу L2-норму весов; таким образом, будем решать задачу гребневой регрессии, Ridge:

$$ \frac{1}{l}(Xw-y)^T(Xw-y) +\gamma||w||_2 \rightarrow \min_{w}. $$

Обучение такой модели в матричном виде.

In [ ]:
class RidgeSGD(BaseEstimator):
    def __init__(self, epsilon=1e-4, max_steps=1000, w0=None, alpha=1e-2, gamma=0):
        """
        epsilon: разница для нормы изменения весов
        max_steps: максимальное количество шагов в градиентном спуске
        w0: np.array (d,) - начальные веса
        alpha: шаг обучения
        gamma: коэффициент регуляризации
        """
        self.epsilon = epsilon
        self.max_steps = max_steps
        self.w0 = w0
        self.alpha = alpha
        self.gamma = gamma
        self.w = None
        self.w_history = []

    def fit(self, X, y):
        """
        X: np.array (l, d)
        y: np.array (l)
        ---
        output: self
        """
        return self

    def predict(self, X):
        """
        X: np.array (l, d)
        ---
        output: np.array (l)
        """
        pass


    def calc_gradient(self, X, y):
        """
        X: np.array (l, d)
        y: np.array (l)
        ---
        output: np.array (d)
        """
        pass

Y_pred_ridge, значение MAPE(y_test, y_pred_ridge).